In [82]:
!pip install matplotlib_inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [88]:
import numpy as np
import scipy.stats as ss
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Ridge, LinearRegression

In [116]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')
import scipy.stats as ss
from scipy import optimize
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint
import pandas as pd
from scipy.integrate import quad
from itertools import product
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

# Ответы

### 1.

Установите np.random.seed(1234) вместо np.random.seed(1337). С чем связано различие в результатах? Что можно сделать, чтобы это различие уменьшилось?

У нас очень маленькая выборка и шум сильно влияет на обучение, нам не повезло с выборкой и все стало очень плохо. Если увеличить размер $n_{obs}$, то наши предсказания будут более устойчивы.

### 2.

Нарисуйте график для a = 5, b = 1. Почему синяя кривая перестала быть близкой к f(x) при x < 0.4, ведь мы знаем, что если обучаемая модель верна (а она верна: $f(x) = 1 + 5x + 0x^2 + 0x^3 = f^*(x, \beta = (1, 5, 0, 0))f(x)=1+5x+0x 
2
 +0x 
3
 =f 
∗
 (x,β=(1,5,0,0)))$, то оценка МНК несмещенная и ее матожидание не зависит от распределения X?

In [ ]:
a, b = 5, 1
x_dist = ss.beta(a, b)
(1 - x_dist.cdf(0.4)) ** 10

0.9021920238235627

Синяя кривая всего лишь оценка посчитанная в каждой точке как:

$$
\frac{\sum_{j=1}^{n_{reps}} f^{*}(\beta^{*}_j, x)}{n_{reps}}
$$

При этом $x$ распределены как $B(5, 1)$ и вероятность того, что все точки в датасете будут $ >0.4$ равна 0.9021. Это значит, что в среднем примерно 900 датасетов из 1000 не содержат ни одной точки меньше $0.4$, и мы никак не контролируем вклад, который делает $f^{*}(\beta^{*}, x)$, обученная на таком датасете.

Тоесть матожидание МНК-оценки несмещено и не зависит от распределения $X$.
Но наша оценка этого матожидания уже зависит.

### 3.

Нарисуйте график для a = 5, b = 1. Почему оценка смещения близка к нулю несмотря на то, что синяя кривая сильно уклоняется от $f(x)$ при $x < 0.4$?

Когда мы оцениваем $\mathbb{E}(bias^2(X))$ мы считаем взвешенные суммы, учитывая изначальное распределение $X$ и вклад $f^{*}$ обученных на плохих датасетах нейтрализуется. Эта оценка выглядит так:

$$
\sum_i w_ibias^2(x_i) = \frac{\sum_{i, j}w_if^{*}(\beta^{*}_j, x_i)}{n_{reps}}
$$

Мы,в принципе, ответы всех моделей на $x_i < 0.4$ учитываем с весом меньше чем $10^{-4}$.

In [ ]:
a, b = 5, 1
x_dist = ss.beta(a, b)
x = np.linspace(0, 1, 1000)
x_ = x[x < 0.4]
x_dist.pdf(x_[-1]) / x_dist.pdf(x).sum()

0.0001270421857323759

### 4.

Нарисуйте график для n_obs = 30. Что изменилось по сравнению с исходным графиком?

Значительно уменьшились $\mathbb{E}(bias^2(X))$ и $\mathbb{E}(variance^2(X))$. Кривые стали точнее описывать нашу прямую: 1 + 5x.

### 5.

Установите степень регуляризации alpha = 0.01. Что изменилось по сравнению с исходным графиком?

Взрывы на концах стали заметно меньше. Теперь мы обучаем кривые, которые точнее описывают прямую.

### 6.

Установите степень регуляризации alpha = 10. Что изменилось по сравнению с исходным графиком? У $x \approx 0$ истинная функция f(x) находится дальше от синей сплошной кривой, чем синяя пунктирная. Что это значит в терминах смещения и дисперсии?

Обученные кривые точнее предсказывают прямую $5x + 1$. Но, когда мы добавляем регуляризацию, МНК-оценка становится смещенной, что мы видим оценив $bias^2(x)$ для $x \approx 0$. При этом $variance$ ожидаемо уменьшается.

# 7. 

Установите степень регуляризации $alpha = 10000$. Почему график так выглядит? К каким значениям стремятся $\mathbb{E} \mathrm{bias}^2(X)$ и $\mathbb{E} \mathrm{variance}(X)$ при $\alpha \to \infty$?

Теперь у нас слишком большой штраф за размер коэффициентов $\beta^{*}$. И все наши $f^{*}(\beta^{*}, x) \approx \overline{Y} \rightarrow 3.5$ .

При $\alpha = \infty$

 $\beta^{*} = (\overline{Y}, 0, 0, 0)$

 $\overline{Y} = \frac{1}{n} \sum (5X_i + 1 + ɛ)$

 Пусть $D \sim ([X]_{10}, [Y]_{10})$ -датасет.

 Тогда 
 $$
 bias^2(x) = (1 + 5x - (1, x, x^2, x^3)\mathbb{E}(β^{*}(D)))^2 = 
 (1 + 5x - \mathbb{E}(\overline{Y}))^2 =
 (1 + 5x - \mathbb{E}(1 + 5X))^2
 \Rightarrow 
 \mathbb{E}(bias^2(x)) = \mathbb{D}(1 + 5X) \approx 2.08
 $$

 Пусть $n = n_{obs}$

 Тогда 

 $$
 \mathbb{D}((1, x, x^2, x^3)\beta^{*}(D))
 = \mathbb{D}(\overline{Y}) =
 \frac{1}{n^2}\sum \mathbb{D}(5X_i + 1 + ɛ) = \frac{1}{n}\mathbb{D}(5X + 1 + ɛ) =
 \frac{1}{n}(\mathbb{D}(5X + 1) + \mathbb{D}(ɛ)) \approx \frac{2.08 + 1}{n} = \frac{3.08}{n}
 $$

# 8.

Измените функцию f(x):

    # Значение матожидания Y, если известно, что X = x
    def f(x):
    ____return 1 + 5 * (x > 0.5)

    
Нарисуйте графики для a = 6, b = 2; a = 4, b = 4; a = 2, b = 6. Объясните различия в синих кривых.

В случаях со скошенной плотностью (a=6, b=2 и a=2 b=6) основное обучение происходит в области высокой плотности x, где f(x) константа. Поэтому мы пытаемся приблизить две различные прымые многочленами 3 степени. А в случае 
a=4, b = 4 основная масса x приходится в окрестность 0.5 и наша кривая пытается 
объяснить разрыв в точке 0.5.

# 9.

Для $f(x)$ из предыдущего пункта нарисуйте график для $n_{obs}$ 10, 100 и 1000. Что меняется? К чему стремится функция $\mathbb{E} f^*(x, \beta^*)$ при $n \to \infty$(явный вид функции считать не надо; если хотите, можете посчитать его численно)?


$$
\mathbb{E} f^*(x, \beta^*) = (1, x, x^2, x^3)\mathbb{E}(\beta^*)
$$

При бесконечной выборке $\beta^*$, такое что $f^*(x, \beta^*)$ лучшая функция в классе $f^*(x, \beta^*) = (1, x, x^2, x^3) \beta^*)$, которая приближает $L^2$
норму функции $f(x)$. Чтобы найти $\mathbb{E}{\beta^*}$ нужно найти:

$$
argmin_{\beta} \int_0^1 (f(x) - f^*(x, \beta))^2 dx
$$

Если минимизировать численно, то получится:
$\beta$ =  (1.93750008 -18.7500046   65.62501378 -43.75000942)

и 

$$
\mathbb{E} f^*(x, \beta^*) \approx 1.93 - 18.75x + 65.62x^2 -43.75x^3
$$

In [126]:
import scipy.integrate as integrate
import scipy.special as special

def f_star(x, beta):
  x_ = np.array([1, x, x**2, x**3])
  return (x_ * beta).sum()

def f(x):
      # Значение матожидания Y, если известно, что X = x
      return 1 + 5 * (x > 0.5)


def mse(beta):
  result = integrate.quad(lambda x: (f(x) - f_star(x, beta))**2, 0, 1)
  return result[0]


bounds=[(None, None) for _ in range(4)]
beta0 = np.zeros(4)

res = minimize(lambda beta: mse(beta),x0=beta0, bounds=bounds)
print('E(beta*) =', res.x)

E(beta*) = [  1.93750008 -18.7500046   65.62501378 -43.75000942]


# 10.


Измените функцию f(x):

    # Значение матожидания Y, если известно, что X = x
    def f(x):
    ___return 1 + 5 * np.sin(np.pi * x)

Почему смещение такое маленькое? Функция $1 + 5\sin(\pi x)$ ведь не является кубической, то есть $f^*(x, \beta)$ ни при каких значениях $\beta$ не равна $f(x)$.

На промежутке $[0,1]$ $1+5sin(πx)$ имеет всего 1 экстремум поэтому она хорошо приближается своим многочленом Тейлора 3-ей степени на всем отрезке, его и приближает наша регрессия.